# Modeling of Random Forest

### This notebook is designed for Random Forest model and its evaluation

### The author of this notebook is Junjie Huang

In [38]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
mpl.style.use('seaborn')
mpl.rcParams['figure.figsize']=(10, 6)
pd.set_option('display.max_columns', None)

### Read the dataset

In [39]:
df = pd.read_csv('train_ver3.csv', index_col='ncodpers')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [40]:
df.head()

,fecha_dato,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
ncodpers,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1375586,2015-01-28,4,0,0,35,2015-01-12,0,6,1,NaN,1,1,1,2,0,0,2,1,29,0,1,87218.100000,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1050611,2015-01-28,4,0,1,23,2012-08-10,0,35,1,NaN,1,2,1,1,0,1,2,1,13,1,0,35548.740000,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1050612,2015-01-28,4,0,1,23,2012-08-10,0,35,1,NaN,1,2,1,2,0,1,2,1,13,1,0,122179.110000,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1050613,2015-01-28,4,0,0,22,2012-08-10,0,35,1,NaN,1,2,1,2,0,2,2,1,50,2,0,119775.540000,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1050614,2015-01-28,4,0,1,23,2012-08-10,0,35,1,NaN,1,1,1,2,0,1,2,1,50,2,1,110587.383626,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
df['fecha_dato'] = pd.to_datetime(df.fecha_dato, format='%Y-%m-%d', errors='coerce')
df['fecha_dato'].unique()

array(['2015-01-28T00:00:00.000000000', '2015-02-28T00:00:00.000000000',
       '2015-03-28T00:00:00.000000000', '2015-04-28T00:00:00.000000000',
       '2015-05-28T00:00:00.000000000', '2015-06-28T00:00:00.000000000',
       '2015-07-28T00:00:00.000000000', '2015-08-28T00:00:00.000000000',
       '2015-09-28T00:00:00.000000000', '2015-10-28T00:00:00.000000000',
       '2015-11-28T00:00:00.000000000', '2015-12-28T00:00:00.000000000',
       '2016-01-28T00:00:00.000000000', '2016-02-28T00:00:00.000000000',
       '2016-03-28T00:00:00.000000000', '2016-04-28T00:00:00.000000000',
       '2016-05-28T00:00:00.000000000'], dtype='datetime64[ns]')

In [42]:
time_list = list(df['fecha_dato'].unique())

### Split the data in May 2015 as training data and the data in May 2016 as testing data

In [45]:
train = df.loc[df.fecha_dato == time_list[4]]
test = df.loc[df.fecha_dato == time_list[-1]]
actual_previous_month_data = df.loc[df.fecha_alta == time_list[-2]] # the data in April 2016

In [46]:
# drop the columns whose data types are datetime
train = train.drop(['ult_fec_cli_1t', 'fecha_dato', 'fecha_alta'], axis=1)
test = test.drop(['ult_fec_cli_1t', 'fecha_dato', 'fecha_alta'], axis=1)
actual_previous_month_data = actual_previous_month_data.drop(['ult_fec_cli_1t', 'fecha_dato', 'fecha_alta'], axis=1)

In [ ]:
# split x_train, y_train, x_test and y_test
x_train = train.iloc[:, :20]
y_train = train.iloc[:, 20:]
x_test = test.iloc[:10000, :20]
y_test = test.iloc[:10000, 20:]
y_test.head(10)

In [9]:
columns_list = y_test.columns
index_list = y_test.index
len(index_list)

10000

### Part of Random Forest Classifier

In [10]:
# set hyperparameters for Random Forest Classifier
rfclf = RandomForestClassifier(n_estimators=50, max_features='sqrt', criterion='entropy', class_weight='balanced', oob_score=True)
rfclf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=None, max_features='sqrt',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [11]:
# prediction probablities
prediction_prob = rfclf.predict_proba(x_test)

In [12]:
prediction_prob[4][3][1]

0.16

In [13]:
product_df = pd.DataFrame(columns=columns_list, index=index_list)
product_df.head()

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
ncodpers,,,,,,,,,,,,,,,,,,,,,,,,
657640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
657788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
657795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
657790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
657794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Create the dataframe for probabilities of products and users
for i in range(len(index_list)):
    for j in range(len(columns_list)):
        product_df.iloc[i,j] = prediction_prob[j][i][1]
product_df.head()            

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
ncodpers,,,,,,,,,,,,,,,,,,,,,,,,
657640,0,0,0.78,0,0,0,0,0.3,0.04,0,0,0,0,0,0,0,0,0.02,0,0,0,0,0,0
657788,0,0,0.92,0,0.06,0,0,0.1,0.08,0,0,0.28,0.28,0.06,0,0.04,0,0.02,0.38,0,0.08,0.04,0.04,0.2
657795,0,0,0.6,0,0.14,0,0,0.12,0.16,0,0,0.1,0.12,0,0,0,0.14,0.14,0.08,0.1,0,0.08,0.08,0.12
657790,0,0,0.8,0,0.16,0,0,0.359871,0.12,0,0,0,0.34,0.02,0,0,0,0.06,0.14,0.019871,0,0.12,0.12,0.46
657794,0,0,0.74,0,0.22,0,0,0.1,0.2,0,0,0.72,0.44,0,0.04,0,0,0.16,0.02,0.58,0.1,0.18,0.18,0.2


In [15]:
# last month products indicator
y_last = df.loc[df.fecha_dato == time_list[-2], 'ind_ahor_fin_ult1':'ind_recibo_ult1']
y_last.head()

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
ncodpers,,,,,,,,,,,,,,,,,,,,,,,,
896836,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
896837,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
896839,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
896853,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
896849,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


In [16]:
# 
for user_id in index_list:
    if user_id in y_last.index:
        for product in columns_list:
            if y_last.loc[user_id, product] == 1:
                product_df.loc[user_id,product] = 0
    if user_id not in y_last.index:
        pass              

In [17]:
product_df.head(10)

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
ncodpers,,,,,,,,,,,,,,,,,,,,,,,,
657640,0,0,0.78,0,0,0,0,0.3,0.04,0,0,0,0,0,0,0,0,0.02,0,0,0,0,0,0
657788,0,0,0,0,0.06,0,0,0.1,0.08,0,0,0.28,0.28,0.06,0,0.04,0,0.02,0.38,0,0.08,0.04,0.04,0.2
657795,0,0,0,0,0.14,0,0,0.12,0.16,0,0,0.1,0.12,0,0,0,0.14,0.14,0.08,0.1,0,0.08,0.08,0.12
657790,0,0,0.8,0,0,0,0,0.359871,0.12,0,0,0,0,0.02,0,0,0,0.06,0,0.019871,0,0,0,0
657794,0,0,0,0,0.22,0,0,0.1,0.2,0,0,0,0.44,0,0.04,0,0,0.16,0.02,0.58,0.1,0.18,0.18,0.2
657797,0,0,0.78,0,0.02,0.02,0,0.16,0,0,0,0,0,0,0,0,0,0.02,0,0,0,0,0,0.02
657789,0,0,0,0,0.54,0,0,0.06,0,0,0,0,0.54,0.04,0,0,0,0.48,0,0.06,0,0.22,0.22,0.3
657787,0,0,0,0,0.04,0,0,0.14,0.04,0,0,0,0.02,0.02,0,0.02,0,0.04,0.02,0.04,0,0.02,0.02,0.06
657777,0,0,0,0,0,0,0,0.08,0,0,0,0,0,0,0,0,0,0.02,0,0,0,0,0,0


In [18]:
threshold = 0.3 # threshold we set
product_dict = {}

In [25]:
# sort the products by their probabilities and add the products whose probablity is larger than threshold to the product list
for user_id in index_list:
    product_dict[user_id] = []
    for product in product_df.loc[user_id].sort_values(ascending=False).index:
        if product_df.loc[user_id, product] >= threshold:
            product_dict[user_id].append(product)

In [26]:
product_dict

In [22]:
df_final = pd.DataFrame(index=product_dict.keys(), columns=['product'])
df_final.index.name = 'ncodpers'
df_final.head()

,product
ncodpers,
657640,NaN
657788,NaN
657795,NaN
657790,NaN
657794,NaN


### Output the final products we recommended

In [23]:
for user_id in index_list:
    df_final.loc[user_id, 'product'] = product_dict[user_id]

In [24]:
df_final

,product
ncodpers,
657640,"[ind_cco_fin_ult1, ind_ctop_fin_ult1]"
657788,[ind_tjcr_fin_ult1]
657795,[]
657790,"[ind_cco_fin_ult1, ind_ctop_fin_ult1]"
657794,"[ind_ecue_fin_ult1, ind_valo_fin_ult1]"
657797,[ind_cco_fin_ult1]
657789,"[ind_cno_fin_ult1, ind_ecue_fin_ult1, ind_reca..."
657787,[]
657777,[]


### Calculate MAP@7 and Hit rate for our prediction recommendations

In [30]:
# validation dataset of products
actual_product = test.iloc[:,20:]
actual_product.head()

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
ncodpers,,,,,,,,,,,,,,,,,,,,,,,,
657640,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
657788,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
657795,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
657790,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,1
657794,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
actual_previous_month_data = actual_previous_month_data.iloc[:, 20:]
actual_product_list = {}

In [49]:
# For the users who are in the validation data, get his actual products list for new products added
actual_product_list = {}
for user_id in product_df.index:
    actual_product_list[user_id] = []
    if user_id in actual_previous_month_data.index:
        for column_name in actual_product.columns:
            if actual_product.loc[user_id, column_name] == 1 and actual_previous_month_data.loc[user_id, column_name] == 0:
                actual_product_list[user_id].append(column_name)
    if user_id not in actual_previous_month_data.index:
        for column_name in actual_product.columns:
            if actual_product.loc[user_id, column_name] == 1:
                actual_product_list[user_id].append(column_name)

In [52]:
# drop the empty recommendations
actual_dict_after_filter = {}
for key, value in actual_product_list.items():
    if len(value) != 0:
        actual_dict_after_filter[key] = value

In [50]:
threshold = 0.01 # set the threshold
product_dict = {}

In [51]:
# sort the products by their probabilities and add the 7 highest products to the product_dict whose probability is larger than threshol
for user_id in index_list:
    product_dict[user_id] = []
    i = 0
    for product in product_df.loc[user_id].sort_values(ascending=False).index:
        if product_df.loc[user_id, product] >= threshold:
            product_dict[user_id].append(product)
            i += 1
            if i>= 7:
                break

In [53]:
# drop the empty recommendations
product_dict_after_filter = {}
for key, value in product_dict.items():
    if len(value) != 0:
        product_dict_after_filter[key] = value

In [54]:
# function of MAP@7 provided by Lu Yin, improved by Junjie Huang
def average_precision(actual, predicted, k=7):
    '''
    Input:    
    actual: the list of actual product for the specific user
    predicted: the list of recommended product for the same user    
    Output: 
    average precision at cutoff 7
    '''
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0
    if len(predicted) == 0 and len(actual) == 0: # if both null values than the average precision is 1. 
        return 1
    elif len(predicted) == 0 and len(actual) != 0:
        return 0
    elif len(predicted) != 0 and len(actual) == 0:
        return 0
    else:
        for i,p in enumerate(predicted):
            if p in actual and p in predicted[:i+1]:
                num_hits += 1.0
                score += num_hits / (i+1) #precision for each step
    if not actual:
        return 0
    return score / min(len(actual), k)

def get_map(actual, predict, k = 7):
    '''
    Input:
    actual: the dictionary of the actual product for the test users
    predict: the dictionary of the recommended product for the test users
    Output:
    MAP@7
    '''
    
    count = 0
    ap = 0
    for key , value in predict.items():
        list1 = value
        if key in actual.keys():
            list2 = actual[key]
            ap+= average_precision(list2, list1)
            count+=1
        elif key not in actual.keys():
            ap+=0
            count+=1
    if count == 0:
        return np.nan
    else:
        result = ap/count
        return result

In [57]:
get_map(actual_product_list, product_dict)

0.006049383219954656

In [58]:
# function of hit rate provided by Lu Yin, improved by Junjie Huang
def get_hit_rate(actual, predict):
    '''
    Input:
    actual: the dictionary of the actual product for the test users
    predict: the dictionary of the recommended product for the test users
    
    Output:
    hit_rate
    '''
    count = 0
    hit_count = 0
    for key , value in predict.items():
        list1 = value
        if key in actual.keys():
            list2 = actual[key]
            common =  set(list1) & set(list2)
            if len(common) > 0:
                hit_count+=1
            if len(list2) != 0: # if the test set is not null. then count it as targeting customers
                count+=1 # if the prediction is not null or test is not null
        if key not in actual.keys():
            hit_count += 0
            count += 1
    if count == 0:
        return np.nan
    else:
        result = hit_count/count
        return result

In [59]:
get_hit_rate(actual_product_list, product_dict)

0.03663855772026752